In [1]:
import h2o
from h2o.automl import H2OAutoML

from sklearn.model_selection import train_test_split

import pandas as pd
import matplotlib.pyplot as plt
from colorama import Fore, Style
import seaborn as sns

rc = {
    "axes.facecolor": "#F8F8F8",
    "figure.facecolor": "#F8F8F8",
    "axes.edgecolor": "#000000",
    "grid.color": "#EBEBE7" + "30",
    "font.family": "serif",
    "axes.labelcolor": "#000000",
    "xtick.color": "#000000",
    "ytick.color": "#000000",
    "grid.alpha": 0.4
}

sns.set_theme(rc=rc)
palette = [
    '#302c36', '#037d97', '#E4591E', '#C09741',
    '#EC5B6D', '#90A6B1', '#6ca957', '#D8E3E2',
]

blk = Style.BRIGHT + Fore.BLACK
mgt = Style.BRIGHT + Fore.MAGENTA
red = Style.BRIGHT + Fore.RED
blu = Style.BRIGHT + Fore.BLUE
res = Style.RESET_ALL

In [2]:
h2o.init()  # Instance lasts for 13 days

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "23.0.1" 2024-10-15; OpenJDK Runtime Environment Homebrew (build 23.0.1); OpenJDK 64-Bit Server VM Homebrew (build 23.0.1, mixed mode, sharing)
  Starting server from /Users/goonzard/.pyenv/versions/3.11.7/envs/baseball/lib/python3.11/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /var/folders/0f/33736c396pgfbzvkb9bc8lnw0000gn/T/tmp2os3od4o
  JVM stdout: /var/folders/0f/33736c396pgfbzvkb9bc8lnw0000gn/T/tmp2os3od4o/h2o_goonzard_started_from_python.out
  JVM stderr: /var/folders/0f/33736c396pgfbzvkb9bc8lnw0000gn/T/tmp2os3od4o/h2o_goonzard_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,Asia/Seoul
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.6
H2O_cluster_version_age:,6 days
H2O_cluster_name:,H2O_from_python_goonzard_d3n1fj
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,1.983 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


# Test on sample data set

* Missing some features

In [3]:
data_df = pd.read_csv("checkup.csv", index_col=0)
data_df.head(10)

,pitcher,batter,ballCount,type,speed,stance,crossPlateX,crossPlateY,topSz,bottomSz,...,az,strike,ball,out,base1,base2,base3,target,event_order,totalBallCount
0,52528,52605,1,투심,142.0,R,-0.078109,0.7083,3.350,1.643,...,-21.2339,0,0,0,0,0,0,1H,1,1
1,52528,52667,1,투심,142.0,R,0.538374,0.7083,3.306,1.621,...,-21.9436,0,0,0,1,0,0,T,2,2
2,52528,52667,2,체인지업,130.0,R,1.240710,0.7083,3.306,1.621,...,-25.0128,1,0,0,1,0,0,S,3,3
3,52528,52667,3,체인지업,132.0,R,0.105212,0.7083,3.306,1.621,...,-25.6861,2,0,0,1,0,0,B,4,4
4,52528,52667,4,체인지업,129.0,R,0.227236,0.7083,3.306,1.621,...,-29.0134,2,1,0,1,0,0,S,5,5
5,52528,64646,1,투심,143.0,R,0.678116,0.7083,3.294,1.616,...,-24.6173,0,0,1,1,0,0,F,6,6
6,52528,64646,2,투심,143.0,R,1.904320,0.7083,3.294,1.616,...,-21.2674,1,0,1,1,0,0,B,7,7
7,52528,64646,3,체인지업,129.0,R,0.211695,0.7083,3.294,1.616,...,-25.1052,1,1,1,1,0,0,F,8,8
8,52528,64646,4,슬라이더,134.0,R,-0.086543,0.7083,3.294,1.616,...,-29.8160,2,1,1,1,0,0,HO,9,9
9,52528,63260,1,투심,144.0,R,1.292990,0.7083,3.407,1.671,...,-19.5171,0,0,2,1,0,0,B,10,10


In [4]:
data_df.describe()

,pitcher,batter,ballCount,speed,crossPlateX,crossPlateY,topSz,bottomSz,vy0,vz0,...,ay,az,strike,ball,out,base1,base2,base3,event_order,totalBallCount
count,334.000000,334.000000,334.000000,334.000000,334.000000,3.340000e+02,334.000000,334.000000,334.000000,334.000000,...,334.000000,334.000000,334.000000,334.000000,334.000000,334.000000,334.000000,334.000000,334.000000,334.000000
mean,56667.988024,60931.473054,2.808383,134.871257,0.036293,7.083000e-01,3.392240,1.663988,-122.688876,-3.475835,...,23.681463,-23.124405,0.853293,0.955090,0.925150,2.673653,1.047904,1.443114,167.500000,21.799401
std,6372.850694,8764.296713,1.630911,8.299836,0.847350,1.111889e-16,0.114029,0.056120,7.491300,2.891743,...,3.565813,6.960194,0.952019,0.983843,0.773294,3.224078,2.316693,2.714890,96.561725,21.091077
min,50662.000000,50150.000000,1.000000,109.000000,-2.531390,7.083000e-01,3.206000,1.572000,-136.468000,-11.703100,...,14.804700,-44.884200,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000
25%,52528.000000,52605.000000,1.000000,129.000000,-0.526967,7.083000e-01,3.304000,1.620000,-129.165750,-5.333018,...,21.331975,-28.067825,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,84.250000,6.000000
50%,52639.000000,62802.000000,2.000000,135.000000,0.047631,7.083000e-01,3.400000,1.668000,-122.690500,-3.587185,...,23.314350,-24.010700,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,167.500000,13.000000
75%,63614.250000,66606.000000,4.000000,142.000000,0.555268,7.083000e-01,3.442000,1.689000,-117.497250,-1.669018,...,26.077200,-17.341675,1.000000,2.000000,2.000000,5.000000,0.000000,2.500000,250.750000,31.750000
max,69620.000000,78513.000000,8.000000,150.000000,2.675600,7.083000e-01,3.636000,1.784000,-99.243600,6.615000,...,35.269000,-8.918640,4.000000,3.000000,2.000000,9.000000,9.000000,9.000000,334.000000,84.000000


In [5]:
feature = data_df.drop('target', axis=1)
label = data_df['target']

In [7]:
label

0      1H
1       T
2       S
3       B
4       S
       ..
329     B
330     F
331     S
332     B
333    HO
Name: target, Length: 334, dtype: object

In [8]:
y_name = 'target'
x_name = list(feature.columns)

train, valid = train_test_split(data_df, test_size=0.2, shuffle=True)
h2o_train = h2o.H2OFrame(train)
h2o_valid = h2o.H2OFrame(valid)

h2o_train[y_name] = h2o_train[y_name].asfactor()
h2o_valid[y_name] = h2o_valid[y_name].asfactor()

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [11]:
# Setup Auto ML Bot
# 5 fold cross validation with balance class if it's unbalanced
# Too few home runs
aml = H2OAutoML(max_runtime_secs=60, seed=1, nfolds=5)
aml.train(x=x_name, y=y_name, training_frame=h2o_train, validation_frame=h2o_valid)

AutoML progress: |
22:54:02.659: User specified a validation frame with cross-validation still enabled. Please note that the models will still be validated using cross-validation only, the validation frame will be used to provide purely informative validation metrics on the trained models.
22:54:02.660: AutoML: XGBoost is not available; skipping it.
22:54:02.665: _train param, Dropping bad and constant columns: [crossPlateY, y0]


22:54:03.289: GLM_1_AutoML_2_20241104_225402 [GLM def_1] failed: DistributedException from /127.0.0.1:54321: 'Index 10 out of bounds for length 10', caused by java.lang.ArrayIndexOutOfBoundsException: Index 10 out of bounds for length 10
22:54:03.291: _train param, Dropping bad and constant columns: [crossPlateY, y0]


22:54:03.664: _train param, Dropping bad and constant columns: [crossPlateY, y0]

█
22:54:04.168: _train param, Dropping bad and constant columns: [crossPlateY, y0]
22:54:04.536: _train param, Dropping bad and constant columns: [crossPlateY, y0

Model Details
=============
H2OGradientBoostingEstimator : Gradient Boosting Machine
Model Key: GBM_grid_1_AutoML_2_20241104_225402_model_8


Model Summary: 
    number_of_trees    number_of_internal_trees    model_size_in_bytes    min_depth    max_depth    mean_depth    min_leaves    max_leaves    mean_leaves
--  -----------------  --------------------------  ---------------------  -----------  -----------  ------------  ------------  ------------  -------------
    21                 210                         450393                 2            15           14.5286       3             217           165.948

ModelMetricsMultinomial: gbm
** Reported on train data. **

MSE: 0.003300426372133251
RMSE: 0.05744933743859237
LogLoss: 0.05545068262178203
Mean Per-Class Error: 0.0
AUC table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).
AUCPR table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class
1H    2H    3H    B    F    HO    HR    S    T    W    Error    Rate
----  ----  ----  ---  ---  ----  ----  ---  ---  ---  -------  -------
15    0     0     0    0    0     0     0    0    0    0        0 / 15
0     1     0     0    0    0     0     0    0    0    0        0 / 1
0     0     2     0    0    0     0     0    0    0    0        0 / 2
0     0     0     114  0    0     0     0    0    0    0        0 / 114
0     0     0     0    36   0     0     0    0    0    0        0 / 36
0     0     0     0    0    32    0     0    0    0    0        0 / 32
0     0     0     0    0    0     1     0    0    0    0        0 / 1
0     0     0     0    0    0     0     23   0    0    0        0 / 23
0     0     0     0    0    0     0     0    42   0    0        0 / 42
0     0     0     0    0    0     0     0    0    1    0        0 / 1
15    1     2     114  36   32    1     23   42   1    0        0 / 267

Top-10 Hit Ratios: 
k    hit_ratio
---  -----------
1    1
2    1
3    1
4    1
5    1
6    1
7    1
8    1
9    1
10   1

ModelMetricsMultinomial: gbm
** Reported on validation data. **

MSE: 0.606600502444873
RMSE: 0.7788456217023197
LogLoss: 2.435172570453501
Mean Per-Class Error: 0.5976190476190476
AUC table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).
AUCPR table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class
1H    2H    3H    B    F    HO    HR    S    T    W    V    Error     Rate
----  ----  ----  ---  ---  ----  ----  ---  ---  ---  ---  --------  -------
0     0     0     1    0    2     0     1    1    0    0    1         5 / 5
0     0     0     0    1    0     0     0    0    0    0    1         1 / 1
0     0     0     0    0    0     0     0    0    0    0    nan       0 / 0
0     0     0     17   1    2     0     3    1    0    0    0.291667  7 / 24
1     0     0     8    2    2     0     0    1    0    0    0.857143  12 / 14
0     0     0     5    1    1     0     0    1    0    0    0.875     7 / 8
0     0     0     0    0    0     0     0    0    0    0    nan       0 / 0
0     0     0     2    0    0     0     1    1    0    0    0.75      3 / 4
2     0     0     1    2    3     0     0    2    0    0    0.8       8 / 10
0     0     0     0    0    0     0     0    0    0    0    nan       0 / 0
0     0     0     1    0    0     0     0    0    0    0    1         1 / 1
3     0     0     35   7    10    0     5    7    0    0    0.656716  44 / 67

Top-10 Hit Ratios: 
k    hit_ratio
---  -----------
1    0.343284
2    0.552239
3    0.671642
4    0.

In [12]:
aml.leaderboard

model_id,mean_per_class_error,logloss,rmse,mse
GBM_grid_1_AutoML_2_20241104_225402_model_8,0.758536,1.7746,0.741568,0.549923
GBM_grid_1_AutoML_2_20241104_225402_model_12,0.79534,1.78892,0.721063,0.519931
GBM_grid_1_AutoML_2_20241104_225402_model_1,0.823872,1.63816,0.727889,0.529822
GBM_5_AutoML_2_20241104_225402,0.82753,1.69798,0.721081,0.519958
DeepLearning_grid_2_AutoML_2_20241104_225402_model_3,0.829844,5.83206,0.717893,0.51537
GBM_grid_1_AutoML_2_20241104_225402_model_4,0.830276,1.61452,0.727459,0.529197
GBM_grid_1_AutoML_2_20241104_225402_model_5,0.83125,1.71152,0.725862,0.526876
GBM_grid_1_AutoML_2_20241104_225402_model_2,0.832177,1.69482,0.721587,0.520688
GBM_grid_1_AutoML_2_20241104_225402_model_10,0.840312,1.65924,0.725648,0.526565
GBM_grid_1_AutoML_2_20241104_225402_model_7,0.841847,1.60501,0.718111,0.515684
